<a href="https://colab.research.google.com/github/caua-sathler/FLPSO-SGD_algorithm/blob/main/FL-DP-PSO-SGD/FL_DP_PSO_SGD_Clients_balanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opacus

In [ ]:
!pip install numpy==1.23.5
!pip install --upgrade scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
from collections import OrderedDict
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus.accountants.utils import get_noise_multiplier
from opacus.accountants import RDPAccountant

In [ ]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [ ]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 25
NUM_RODADAS = 10
NUM_DIGITOS = 10
INERCIA, C1, C2 = 0.9, 0.8,  0.9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 240
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE)
criterio = nn.CrossEntropyLoss()

training on cuda


In [ ]:
#Seeds para reprodutibilidade
numero = random.randint(70, 140)
print(numero)
random.seed(numero)
torch.manual_seed(numero)
torch.cuda.manual_seed(numero)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        #self.pesos = copy.deepcopy(modelo_cliente.state_dict())
        self.pesos = {f"_module.{key}": value.clone() for key, value in modelo_cliente.state_dict().items()}
        self.device = modelo_cliente.device

        # Adiciona ruído leve nos pesos para quebrar simetria inicial
        for name in self.pesos:
            self.pesos[name] += 0.01 * torch.randn_like(self.pesos[name])
            #self.pesos[name] += 0.001 * torch.randn_like(self.pesos[name])

        self.melhor_pesos = copy.deepcopy(self.pesos)
        self.melhor_erro = float('inf')
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}


    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2):
        MAX_VELOCITY = 0.1  # Limite para evitar oscilações grandes
        if(round == 0):
          global_best_weights_adjusteds = {f"_module.{key}": value for key, value in global_best_pesos.items()}
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name]
                + C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name])
                + C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )

            # Clipping da velocidade
            self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)

            # Atualiza os pesos com a nova velocidade
            self.pesos[name] += self.velocidade[name]

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        modelo_cliente.load_state_dict(self.pesos)
        modelo_cliente.eval()
        total_loss = 0
        device = next(modelo_cliente.parameters()).device

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()


        return total_loss / len(dados)



class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, test, num_particulas):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.test = test
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)
        self.optimizer = optim.Adam(self.modelo.parameters(), lr=0.005, weight_decay=1e-5)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:
            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()
        # modelo_global.load_state_dict(self.melhor_particula.pesos)

    def refinar_com_adam(self, criterio):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        # self.modelo.load_state_dict(modelo_global.state_dict())
        device = next(self.modelo.parameters()).device

        self.modelo.train()
        for i in range(1):
          with BatchMemoryManager(data_loader=self.dados, max_physical_batch_size=BATCH_SIZE, optimizer=self.optimizer) as new_data_loader:
            for inputs, labels in new_data_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.modelo(inputs)
                loss = criterio(outputs, labels)
                loss.backward()
                self.optimizer.step()
        self.melhor_particula.melhor_pesos = copy.deepcopy(self.modelo.state_dict())
        self.melhor_particula.melhor_erro = self.calcular_loss(self.modelo, criterio, self.dados)

    def calcular_loss(self, modelo, criterio, dados):
        self.modelo.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = modelo(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = {f"_module.{key}": value.clone() for key, value in modelo_global.state_dict().items()}
    melhor_erro_global = float('inf')
    soma = 0.0;
    for rodada in range(num_rodadas):
        resultados_rodada = []

        for cliente in clientes:
          cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio)  # Treino com PSO
          cliente.refinar_com_adam(criterio)  # Refinamento com Adam
          erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
          pesos_cliente = cliente.melhor_particula.melhor_erro  # Obtém os pesos do modelo do cliente
          resultados_rodada.append((cliente.cliente_id, erro_cliente))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        melhor_cliente = random.choice(top_3_results)
        melhor_cliente_id = melhor_cliente[0]
        melhor_erro_cliente = melhor_cliente[1]

        melhor_peso_global = copy.deepcopy(clientes[melhor_cliente_id].melhor_particula.melhor_pesos)
        melhor_peso_global_adjusted =  {key.replace("_module.", ""): value for key, value in melhor_peso_global.items()}
        melhor_erro_global = melhor_erro_cliente

        modelo_global.load_state_dict(melhor_peso_global_adjusted)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)
        soma += test_accuracy

        # if (rodada+1) % 10 == 0:
        print(f"Rodada {rodada+1}/{num_rodadas}: Cliente {melhor_cliente_id} enviou os pesos!")
        print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print("Epsilon 5 balanceado")
        print(f"Teste -> Perda: {test_loss: .4f}, Acurácia: {test_accuracy: .4f}\n")

    print(f"Acurácia média: {soma/num_rodadas: .2f}")
    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples)

    return test_loss, test_accuracy


mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten
y_test = mnist_test.targets.numpy()

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten
y_train = mnist_train.targets.numpy()

# Normalização (como foi feito com o Iris)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)

def create_stratified_subsets(dataset, num_clients):
    # Extrair os labels do dataset
    targets = dataset.tensors[1].numpy()

    # Dicionário para armazenar os índices de cada classe (0 a 9)
    class_indices = {}
    for digit in range(10):
        indices = np.where(targets == digit)[0]
        np.random.shuffle(indices)  # Embaralha os índices de cada classe
        class_indices[digit] = indices

    # Dicionário para armazenar os índices para cada cliente
    client_indices = {i: [] for i in range(num_clients)}

    # Para cada classe, divide os índices igualmente entre os clients
    for digit in range(10):
        indices = class_indices[digit]
        splits = np.array_split(indices, num_clients)
        for client_id in range(num_clients):
            client_indices[client_id].extend(splits[client_id].tolist())

    # Cria os subsets utilizando os índices calculados
    subsets = [Subset(dataset, client_indices[i]) for i in range(num_clients)]
    return subsets

train_subsets = create_stratified_subsets(trainset, NUM_CLIENTES)


trainloaders = [DataLoader(train_subsets[i], batch_size=BATCH_SIZE, shuffle=True) for i in range(NUM_CLIENTES)]

testloader = DataLoader(testset, batch_size=32, shuffle=False)
# Criar DataLoaders para cada subset (cliente)
trainloaders = [DataLoader(train_subsets[i], batch_size=240, shuffle=True) for i in range(NUM_CLIENTES)]

testloader = DataLoader(testset, batch_size=240, shuffle=False)

clientes = [Cliente(i, modelo_global, trainloaders[i], testloader, NUM_PARTICULAS) for i in range(NUM_CLIENTES)]

privacy_engines = [PrivacyEngine() for i in range(NUM_CLIENTES)]

for i in range (NUM_CLIENTES):
    clientes[i].modelo, clientes[i].optimizer, clientes[i].dados = privacy_engines[i].make_private_with_epsilon(
    module=clientes[i].modelo,
    optimizer=clientes[i].optimizer,
    data_loader=clientes[i].dados,
    epochs = NUM_RODADAS,
    target_epsilon = 5,
    target_delta = 1e-5,
    max_grad_norm=1.0
)


# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader)
for i in range(NUM_CLIENTES):
  print(privacy_engines[i].get_epsilon(delta = 1e-5))

133


/usr/local/lib/python3.11/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Rodada 1/10: Cliente 2 enviou os pesos!
Erro Global Atualizado: 0.5315
Epsilon 5 balanceado
Teste -> Perda:  0.5392, Acurácia:  0.8502

Rodada 2/10: Cliente 1 enviou os pesos!
Erro Global Atualizado: 0.4769
Epsilon 5 balanceado
Teste -> Perda:  0.4725, Acurácia:  0.8835

Rodada 3/10: Cliente 1 enviou os pesos!
Erro Global Atualizado: 0.4215
Epsilon 5 balanceado
Teste -> Perda:  0.4487, Acurácia:  0.8944

Rodada 4/10: Cliente 3 enviou os pesos!
Erro Global Atualizado: 0.4307
Epsilon 5 balanceado
Teste -> Perda:  0.4646, Acurácia:  0.8874

Rodada 5/10: Cliente 3 enviou os pesos!
Erro Global Atualizado: 0.4395
Epsilon 5 balanceado
Teste -> Perda:  0.4521, Acurácia:  0.8986

Rodada 6/10: Cliente 2 enviou os pesos!
Erro Global Atualizado: 0.4019
Epsilon 5 balanceado
Teste -> Perda:  0.4184, Acurácia:  0.8975

Rodada 7/10: Cliente 4 enviou os pesos!
Erro Global Atualizado: 0.4033
Epsilon 5 balanceado
Teste -> Perda:  0.4124, Acurácia:  0.9027

Rodada 8/10: Cliente 3 enviou os pesos!
Erro Glo

In [ ]:

# accuracies = [0.9375, 0.9356, 0.9315, 0.9345, 0.9259, 0.9315, 0.9340,  0.9304, 0.8951, 0.9366]
# mean_acc = np.mean(accuracies)
# std_acc = np.std(accuracies)

# print(f"Acurácia média: {mean_acc * 100:.2f}% ± {std_acc * 100:.2f}%")